In [ ]:
# | default_exp environments.basic

In [ ]:
# | hide
from nbdev.showdoc import *

In [ ]:
# | export

import gymnasium as gym
from gymnasium import spaces
import numpy as np


class SimpleTradingEnv(gym.Env):
    """
    A basic Gymnasium environment for trading simulation.

    This environment simulates trading a single cryptocurrency using a simplified action space
    and observation space. It's designed to be a starting point for reinforcement learning
    experiments in crypto trading and prioritizes clarity and ease of use.

    Key Features:
        - Discrete action space: Buy, Sell, Hold.
        - Observation space: Current price, crypto held, cash balance.
        - Simple reward function: Portfolio value change.
        - Basic transaction cost simulation.
        - Clear comments and adherence to best software practices.

    Important Notes:
        - This is a *simplified* environment. It does not include many real-world complexities
          like order book dynamics, slippage (beyond basic transaction cost), market impact,
          multiple cryptocurrencies, or external factors influencing price.
        - The price data is pre-loaded and deterministic, making it suitable for initial
          algorithm testing and development but not for realistic backtesting or live trading.
        - For more advanced and realistic simulations, consider building upon this foundation
          or using more sophisticated trading environment libraries.

    Best Practices Implemented:
        - **Clear Class and Method Docstrings:** Explaining the purpose and usage of each component.
        - **Type Hinting:** Improving code readability and helping catch type-related errors.
        - **Meaningful Variable Names:** Enhancing code understanding.
        - **Modular Structure:**  Methods for `step`, `reset`, `render`, etc., as per Gym standards.
        - **Configuration via `__init__`:**  Allowing customization of environment parameters.
        - **Error Handling (Basic):**  Checking for invalid actions (e.g., selling without crypto).
        - **Concise and Focused Code:**  Avoiding unnecessary complexity for a basic environment.
    """

    metadata = {
        "render.modes": ["human", "rgb_array"]
    }  # Optional rendering modes (currently only 'human' for text output)

    def __init__(
        self,
        price_data: np.ndarray,
        initial_cash: float = 10000,
        transaction_cost_percent: float = 0.001,
    ):
        """
        Initializes the CryptoTradingEnv environment.

        Args:
            price_data (np.ndarray): 1D numpy array of historical cryptocurrency prices.
                                      Each element represents the price at a time step.
            initial_cash (float, optional): Starting cash balance. Defaults to 10000.
            transaction_cost_percent (float, optional): Transaction cost as a percentage of trade value. Defaults to 0.001 (0.1%).
        """
        super(
            CryptoTradingEnv, self
        ).__init__()  # Call superclass constructor to initialize gym.Env

        # --- Input Validation and Data Storage ---
        if not isinstance(price_data, np.ndarray) or price_data.ndim != 1:
            raise ValueError("price_data must be a 1D numpy array.")
        if not np.issubdtype(price_data.dtype, np.number):
            raise ValueError("price_data must contain numerical values (prices).")
        if initial_cash <= 0:
            raise ValueError("initial_cash must be a positive value.")
        if (
            not 0 <= transaction_cost_percent < 1
        ):  # Transaction cost should be between 0 and 1 (exclusive of 1)
            raise ValueError(
                "transaction_cost_percent must be between 0 and 1 (exclusive of 1)."
            )

        self.price_data = price_data
        self.initial_cash = initial_cash
        self.transaction_cost_percent = transaction_cost_percent

        # --- Define Action and Observation Space ---
        # Action space: Discrete 3 actions: 0 = Hold, 1 = Buy, 2 = Sell
        self.action_space = spaces.Discrete(3)
        # Observation space: Dictionary containing 'price', 'crypto_held', 'cash_balance'
        # We use Box for continuous values (price, cash, crypto holding - although crypto_held could be discrete if we only trade in whole units, but for flexibility we keep it continuous)
        self.observation_space = spaces.Dict(
            {
                "price": spaces.Box(
                    low=0, high=np.inf, shape=(1,), dtype=np.float32
                ),  # Current price (single value)
                "crypto_held": spaces.Box(
                    low=0, high=np.inf, shape=(1,), dtype=np.float32
                ),  # Amount of cryptocurrency held
                "cash_balance": spaces.Box(
                    low=0, high=np.inf, shape=(1,), dtype=np.float32
                ),  # Current cash balance
            }
        )

        # --- Initialize Environment State ---
        self.current_step = 0  # Track the current time step in the price data
        self.cash_balance = initial_cash
        self.crypto_held = 0.0  # Start with no cryptocurrency
        self.portfolio_value = (
            initial_cash  # Initial portfolio value is just the initial cash
        )
        self.net_worth_history = [
            initial_cash
        ]  # Keep track of net worth over time for analysis/rendering

        # --- Placeholder for rendering (can be expanded for more sophisticated visualization) ---
        self.renderer = None  # Placeholder for a renderer object if you want to add visual rendering

    def reset(self, seed=None, options=None):
        """
        Resets the environment to its initial state.

        This is called at the beginning of each episode.

        Args:
            seed (int, optional): Seed for random number generator (not used in this deterministic env but included for Gym API compliance). Defaults to None.
            options (dict, optional): Options for reset (not used in this basic env but included for Gym API compliance). Defaults to None.

        Returns:
            tuple: Initial observation and info dictionary.
                   - observation (dict): Initial observation dictionary as defined by self.observation_space.
                   - info (dict):  Additional information (can be empty in this basic env).
        """
        super().reset(
            seed=seed, options=options
        )  # Call superclass reset for seed handling (even though not used here)

        self.current_step = 0
        self.cash_balance = self.initial_cash
        self.crypto_held = 0.0
        self.portfolio_value = self.initial_cash
        self.net_worth_history = [self.initial_cash]

        observation = self._get_observation()  # Get initial observation
        info = {}  # Empty info dictionary for now

        return observation, info

    def _get_observation(self):
        """
        Helper function to construct the observation dictionary for the current state.

        Returns:
            dict: Observation dictionary containing 'price', 'crypto_held', and 'cash_balance'.
        """
        current_price = self.price_data[self.current_step]
        return {
            "price": np.array(
                [current_price], dtype=np.float32
            ),  # Ensure price is a numpy array of shape (1,) and float32
            "crypto_held": np.array(
                [self.crypto_held], dtype=np.float32
            ),  # Ensure crypto_held is a numpy array of shape (1,) and float32
            "cash_balance": np.array(
                [self.cash_balance], dtype=np.float32
            ),  # Ensure cash_balance is a numpy array of shape (1,) and float32
        }

    def step(self, action: int):
        """
        Executes one time step within the environment.

        This function takes an action, updates the environment state based on the action and price data,
        and returns the next observation, reward, termination status, truncation status, and info dictionary.

        Args:
            action (int): Action to take. Must be one of:
                          0 = Hold, 1 = Buy, 2 = Sell.

        Returns:
            tuple: observation, reward, terminated, truncated, info
                   - observation (dict): Next observation dictionary.
                   - reward (float): Reward obtained in this step.
                   - terminated (bool): True if the episode is terminated (e.g., end of price data).
                   - truncated (bool): True if the episode is truncated (not used in this basic env, always False).
                   - info (dict):  Additional information (can be empty in this basic env).
        """
        # --- Input Validation ---
        if action not in [0, 1, 2]:
            raise ValueError(
                f"Invalid action: {action}. Action must be in [0, 1, 2] (Hold, Buy, Sell)."
            )

        current_price = self.price_data[self.current_step]
        prev_portfolio_value = (
            self.portfolio_value
        )  # Store previous portfolio value to calculate reward

        # --- Execute Action ---
        transaction_cost = 0  # Initialize transaction cost for this step

        if action == 1:  # Buy
            # Buy with all available cash (for simplicity in this basic env)
            amount_to_buy = self.cash_balance
            if amount_to_buy > 0:  # Ensure we have cash to buy
                transaction_cost = amount_to_buy * self.transaction_cost_percent
                crypto_to_buy = (amount_to_buy - transaction_cost) / current_price
                self.crypto_held += crypto_to_buy
                self.cash_balance -= amount_to_buy
        elif action == 2:  # Sell
            # Sell all crypto held (for simplicity in this basic env)
            amount_to_sell = self.crypto_held * current_price
            if amount_to_sell > 0:  # Ensure we have crypto to sell
                transaction_cost = amount_to_sell * self.transaction_cost_percent
                cash_from_sale = amount_to_sell - transaction_cost
                self.cash_balance += cash_from_sale
                self.crypto_held = 0.0
        elif action == 0:  # Hold
            pass  # Do nothing, just hold current positions

        # --- Update Environment State ---
        self.current_step += 1

        # Calculate portfolio value at the end of the step (cash + value of crypto holdings)
        self.portfolio_value = self.cash_balance + self.crypto_held * current_price
        self.net_worth_history.append(self.portfolio_value)

        # --- Calculate Reward ---
        # Simple reward: change in portfolio value
        reward = self.portfolio_value - prev_portfolio_value

        # --- Check for Termination ---
        terminated = (
            self.current_step >= len(self.price_data) - 1
        )  # End episode when we reach the end of price data
        truncated = False  # Truncation is not used in this basic environment

        # --- Get Next Observation and Info ---
        observation = self._get_observation()
        info = {
            "step": self.current_step,
            "price": current_price,
            "transaction_cost": transaction_cost,
            "action": action,  # Useful for debugging and analysis
            "portfolio_value": self.portfolio_value,
            "cash_balance": self.cash_balance,
            "crypto_held": self.crypto_held,
            "reward": reward,
            # Add any other relevant information you want to track in 'info'
        }

        return observation, reward, terminated, truncated, info

    def render(self, mode="human"):
        """
        Renders the environment.

        In 'human' mode, it prints the current state to the console.
        For 'rgb_array' mode (not fully implemented here), it should return an RGB array representing the environment.

        Args:
            mode (str, optional): Rendering mode. 'human' (text output) or 'rgb_array' (not fully implemented). Defaults to 'human'.
        """
        if mode == "human":
            print(f"------------------ Step: {self.current_step} ------------------")
            print(f"Price: ${self.price_data[self.current_step]:.2f}")
            print(
                f"Action: {self._get_action_name(self.step_history[-1]['action'] if hasattr(self, 'step_history') and self.step_history else 'Initial')}"
            )  # Access last action from info if available
            print(f"Crypto Held: {self.crypto_held:.4f}")
            print(f"Cash Balance: ${self.cash_balance:.2f}")
            print(f"Portfolio Value: ${self.portfolio_value:.2f}")
            print(
                f"Reward: {self.step_history[-1]['reward']:.4f}"
                if hasattr(self, "step_history") and self.step_history
                else "Initial"
            )  # Access last reward from info if available
            print("----------------------------------------------------")
        elif mode == "rgb_array":
            # For more advanced rendering (e.g., using matplotlib to plot portfolio value), you would
            # generate an image as a numpy array here and return it.
            # This is a placeholder for future expansion.
            super().render_modes = [
                "rgb_array",
                "human",
            ]  # Define supported render modes
            return (
                self._render_frame()
            )  # You would need to implement _render_frame to return an RGB array
        else:
            super().render_modes = [
                "rgb_array",
                "human",
            ]  # Define supported render modes
            super().render()  # Call default render method if mode is not recognized

    def _get_action_name(self, action_code):
        """Helper function to get action name from action code for rendering."""
        if action_code == 0:
            return "Hold"
        elif action_code == 1:
            return "Buy"
        elif action_code == 2:
            return "Sell"
        else:
            return "Unknown"

    def close(self):
        """
        Closes the environment and performs any necessary cleanup.

        In this basic environment, there's no specific cleanup needed, but this method is included for Gym API compliance
        and for potential future expansion where resources might need to be released.
        """
        if self.renderer:
            self.renderer.close()  # If you add a renderer, close it here
        super().close()  # Call superclass close method

In [ ]:
# | export
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
from typing import Tuple, Optional, Union


class CryptoTradingEnv(gym.Env):
    """
    A cryptocurrency trading environment for reinforcement learning.

    This environment simulates trading a single cryptocurrency based on historical price data.
    It supports buying, selling, and holding actions, and provides observations
    that can include price history, technical indicators, and the agent's current holdings.

    Key Concepts:
        -   Action Space:  Discrete (buy, sell, hold).  Could be extended to continuous.
        -   Observation Space: Box (multidimensional array).  Shape depends on features.
        -   Reward Function:  Change in portfolio value (can be customized).
        -   Episode Termination:  End of data or when the agent runs out of funds.
        -   Data: Requires a pandas DataFrame with 'close' prices (and optionally other features).

    """

    metadata = {
        "render_modes": ["human"],
        "render_fps": 30,
    }  # For future rendering capabilities

    def __init__(
        self,
        data: pd.DataFrame,
        initial_balance: float = 10000.0,
        transaction_fee_percent: float = 0.001,  # 0.1% transaction fee
        lookback_window_size: int = 50,  # Number of past timesteps to include in observation
        features: list[str] = ["close"],  # List of columns to use as features
        render_mode: Optional[str] = None,
    ) -> None:
        """
        Initializes the CryptoTradingEnv.

        Args:
            data (pd.DataFrame):  Historical price data (and other features).  MUST include a 'close' column.
            initial_balance (float): Starting capital for the agent.
            transaction_fee_percent (float):  Fee percentage per transaction (e.g., 0.001 for 0.1%).
            lookback_window_size (int):  How many past timesteps to use in the observation.
            features (List[str]):  Column names from 'data' to use as observation features.
            render_mode (str, optional): Render mode for human visualization. Defaults to None.
        Raises:
            ValueError: If input data is invalid or required columns are missing.
        """
        super().__init__()

        if not isinstance(data, pd.DataFrame):
            raise ValueError("Data must be a pandas DataFrame.")
        if "close" not in data.columns:
            raise ValueError("Data must contain a 'close' column for price data.")
        if not all(feature in data.columns for feature in features):
            raise ValueError("All features must be present in the data columns.")
        if lookback_window_size <= 0:
            raise ValueError("lookback_window_size must be greater than 0.")

        self.data = data
        self.features = features
        self.initial_balance = initial_balance
        self.transaction_fee_percent = transaction_fee_percent
        self.lookback_window_size = lookback_window_size
        self.render_mode = render_mode

        # Action Space: 0 = Hold, 1 = Buy, 2 = Sell
        self.action_space = spaces.Discrete(3)

        # Observation Space: (lookback_window_size, num_features) + balance + holdings
        self.observation_space = spaces.Box(
            low=-np.inf,
            high=np.inf,
            shape=(lookback_window_size, len(self.features) + 2),
            dtype=np.float32,
        )

        self.current_step = lookback_window_size
        self.balance = initial_balance
        self.holdings = 0.0  # Amount of crypto held
        self.portfolio_value = initial_balance  # balance + (holdings * current_price)
        self._done = False  # added to track done state explicitly

    def _get_observation(self) -> np.ndarray:
        """
        Constructs the observation array.

        Returns:
            np.ndarray:  The observation array.
        """
        window_data = self.data[self.features][
            self.current_step - self.lookback_window_size : self.current_step
        ].values

        # Add balance and holdings to the observation.  Reshape for consistent shape.
        balance_and_holdings = np.array([[self.balance, self.holdings]])
        repeated_balance_holdings = np.repeat(
            balance_and_holdings, self.lookback_window_size, axis=0
        )
        observation = np.concatenate((window_data, repeated_balance_holdings), axis=1)

        return observation.astype(np.float32)

    def _calculate_reward(self, action: int) -> float:
        """
        Calculates the reward based on the change in portfolio value.

        Args:
            action (int): The action taken by the agent.

        Returns:
            float: The reward for the current step.  (Can be customized).
        """
        current_price = self.data["close"][self.current_step]
        previous_portfolio_value = self.portfolio_value
        self.portfolio_value = self.balance + (self.holdings * current_price)
        reward = self.portfolio_value - previous_portfolio_value
        return reward

    def _execute_trade(self, action: int) -> None:
        """
        Executes a trade (buy or sell) and updates balance/holdings.

        Args:
            action (int): The action taken (1 for buy, 2 for sell).
        """

        current_price = self.data["close"][self.current_step]

        if action == 1:  # Buy
            affordable_amount = self.balance / current_price
            cost = affordable_amount * current_price
            fee = cost * self.transaction_fee_percent
            if self.balance >= cost + fee:
                self.balance -= cost + fee
                self.holdings += affordable_amount * (1 - self.transaction_fee_percent)

        elif action == 2:  # Sell
            fee = self.holdings * current_price * self.transaction_fee_percent
            self.balance += self.holdings * current_price - fee
            self.holdings = 0.0

    def step(self, action: int) -> Tuple[np.ndarray, float, bool, bool, dict]:
        """
        Takes a step in the environment.

        Args:
            action (int): The action to take (0, 1, or 2).

        Returns:
            Tuple: A tuple containing:
                - observation (np.ndarray): The next state's observation.
                - reward (float): The reward for the action.
                - terminated (bool): Whether the episode has ended.
                - truncated (bool): Whether the episode was truncated (not used here, but required by gym API).
                - info (dict): Additional information (empty in this case).
        """
        if self._done:
            # If the episode is already done, you can either reset it or raise an error.
            # Here we reset.  Alternatively, you can raise:  raise Exception("Episode already done")
            return self.reset()[0], 0.0, True, False, {}

        self._execute_trade(action)  # Execute the trade first
        reward = self._calculate_reward(action)  # Calculate reward *after* trade

        self.current_step += 1
        # Check if the episode is done (either end of data or out of money)
        self._done = self.current_step >= len(self.data) - 1 or self.balance <= 0
        truncated = False
        observation = self._get_observation()
        info: dict = {}
        return observation, reward, self._done, truncated, info  # type: ignore

    def reset(
        self, *, seed: Optional[int] = None, options: Optional[dict] = None
    ) -> Tuple[np.ndarray, dict]:
        """
        Resets the environment to the initial state.

        Args:
            seed (Optional[int]): Seed for random number generation.
            options (Optional[dict]):  Additional options (not used here).

        Returns:
            Tuple: A tuple containing:
                - observation (np.ndarray): The initial observation.
                - info (dict):  Additional info (empty in this case).
        """
        super().reset(seed=seed)  # Call gym.Env's reset with the seed

        self.current_step = self.lookback_window_size
        self.balance = self.initial_balance
        self.holdings = 0.0
        self.portfolio_value = self.initial_balance
        self._done = False  # Reset the done flag
        observation = self._get_observation()
        info: dict = {}
        return observation, info

    def render(self) -> None:
        """
        Renders the environment (not implemented here, but could be used for visualization).
        """
        # Future implementation for visualization (e.g., using matplotlib).
        if self.render_mode == "human":
            print(
                f"Step: {self.current_step}, Balance: {self.balance:.2f}, "
                f"Holdings: {self.holdings:.4f}, Portfolio Value: {self.portfolio_value:.2f}"
            )

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()